In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import torch

In [2]:
import HighLevelFeatures as HLF

def extract_shower_and_energy(given_file, which):
    """ reads .hdf5 file and returns samples and their energy """
    print("Extracting showers from {} file ...".format(which))
    if which == 0.:
        shower = given_file['showers'][:]
        energy = given_file['incident_energies'][:]
    else:
        shower = given_file['showers'][:]
        energy = given_file['incidence energy'][:]
    print("Extracting showers from {} file: DONE.\n".format(which))
    return shower, energy

def prepare_high_data_for_classifier(hdf5_file, hlf_class, label):
    """ takes hdf5_file, extracts high-level features, appends label, returns array """
    voxel, E_inc = extract_shower_and_energy(hdf5_file, label)
    E_tot = hlf_class.GetEtot()
    E_layer = []
    for layer_id in hlf_class.GetElayers():
        E_layer.append(hlf_class.GetElayers()[layer_id].reshape(-1, 1))
    EC_etas = []
    EC_phis = []
    Width_etas = []
    Width_phis = []
    for layer_id in hlf_class.layersBinnedInAlpha:
        EC_etas.append(hlf_class.GetECEtas()[layer_id].reshape(-1, 1))
        EC_phis.append(hlf_class.GetECPhis()[layer_id].reshape(-1, 1))
        Width_etas.append(hlf_class.GetWidthEtas()[layer_id].reshape(-1, 1))
        Width_phis.append(hlf_class.GetWidthPhis()[layer_id].reshape(-1, 1))
    E_layer = np.concatenate(E_layer, axis=1)
    EC_etas = np.concatenate(EC_etas, axis=1)
    EC_phis = np.concatenate(EC_phis, axis=1)
    Width_etas = np.concatenate(Width_etas, axis=1)
    Width_phis = np.concatenate(Width_phis, axis=1)
    ret = np.concatenate([np.log10(E_inc), np.log10(E_layer+1e-8), EC_etas/1e2, EC_phis/1e2,
                          Width_etas/1e2, Width_phis/1e2, label*np.ones_like(E_inc)], axis=1)
    return ret

In [3]:
test_data = h5py.File('/fast_scratch_1/caloqvae/test_data/dataset_2_2.hdf5', 'r')
gen_data = h5py.File("/fast_scratch_1/caloqvae/syn_data/dataset2_synthetic_denim-smoke-166en130.hdf5", 'r')

In [4]:
data_showers = (np.array(test_data['showers']))
energy = (np.array(test_data['incident_energies']))

In [5]:
gen_showers = (np.array(gen_data['showers']))

In [6]:
hlf = HLF.HighLevelFeatures('electron', filename='/fast_scratch_1/caloqvae/data/atlas_dataset2and3/binning_dataset_2.xml', wandb=False)
ref_hlf = HLF.HighLevelFeatures('electron', filename='/fast_scratch_1/caloqvae/data/atlas_dataset2and3/binning_dataset_2.xml', wandb=False)

In [7]:
hlf.CalculateFeatures(data_showers)
ref_hlf.CalculateFeatures(gen_showers)
hlf.Einc = energy

In [8]:
hlf_test_data = prepare_high_data_for_classifier(test_data, hlf, 0.)[:, :-1]
hlf_gen_data = prepare_high_data_for_classifier(gen_data, ref_hlf, 1.)[:, :-1]

Extracting showers from 0.0 file ...
Extracting showers from 0.0 file: DONE.

Extracting showers from 1.0 file ...
Extracting showers from 1.0 file: DONE.



In [17]:
print(hlf_test_data, hlf_test_data.shape)
print(hlf_gen_data, hlf_gen_data.shape)

[[5.87222238e+00 2.14741203e+00 2.60703610e+00 ... 1.46077735e-01
  1.54922758e-01 1.87796553e-01]
 [5.64951388e+00 2.64935583e+00 3.12108231e+00 ... 1.75352528e-01
  1.18627905e-01 1.85484302e-01]
 [4.71030195e+00 1.90820993e+00 2.26416288e+00 ... 5.69447023e-02
  0.00000000e+00 0.00000000e+00]
 ...
 [4.48007254e+00 1.77803682e+00 2.27939870e+00 ... 2.38418579e-09
  2.38418579e-09 1.03238273e-09]
 [5.80095663e+00 2.84187862e+00 3.25197880e+00 ... 1.64515871e-01
  1.31542148e-01 9.96026703e-02]
 [4.36830676e+00 1.41235059e+00 1.85547142e+00 ... 0.00000000e+00
  0.00000000e+00 2.38418579e-09]] (100000, 226)
[[3.24379802 1.68895793 1.94245017 ... 0.         0.         0.        ]
 [5.69856596 3.34499073 3.45755553 ... 0.17346888 0.19187891 0.19636657]
 [5.18363762 2.7730341  2.93820977 ... 0.         0.         0.        ]
 ...
 [5.60514641 3.0929606  3.08861208 ... 0.15330487 0.17804461 0.15050746]
 [3.17415452 1.3306849  1.90820003 ... 0.         0.         0.        ]
 [3.06647444 1.5

In [10]:
import jetnet

In [12]:
fpd_val, fpd_err = jetnet.evaluation.fpd(hlf_test_data, hlf_gen_data)
kpd_val, kpd_err = jetnet.evaluation.kpd(hlf_test_data, hlf_gen_data)

result_str = (
    f"FPD (x10^3): {fpd_val*1e3:.4f} ± {fpd_err*1e3:.4f}\n" 
    f"KPD (x10^3): {kpd_val*1e3:.4f} ± {kpd_err*1e3:.4f}"
)

print(result_str)

# norm -0.008

FPD (x10^3): 1469.7943 ± 8.3248
KPD (x10^3): 15.6822 ± 1.2243


In [15]:
norm = 0.008
print(fpd_val - norm)
print(kpd_val)

1.461794261314703
0.01568219628651657
